In [1]:
import os

import pandas as pd
import numpy as np

import re
import unidecode

In [2]:
# Crear una función para extraer colonia y CP
def extraer_colonia_cp(direccion, df_cp):
    for _, row in df_cp.iterrows():
        if row['d_asenta'] in direccion:
            return row['d_asenta'], row['d_codigo']
    return None, None

In [3]:
# Estandarizar texto: quitar acentos, convertir a minúsculas, eliminar espacios extra y reemplazar "\n" por " "
def estandarizar_texto(texto):
    if pd.isna(texto):
        return ""
    texto = texto.replace("\n", " ")  # Reemplazar saltos de línea por espacios
    texto = texto.replace(",", "")  # Quitar comas
    return unidecode.unidecode(texto.strip().lower())

### Read webscrapping data

In [4]:
df_web = pd.read_parquet('../../data/processed/inmuebles24_departamentos_20251005.parquet')

In [5]:
df_web

,precio_mxn,lote_m2,recamaras,baños,estacionamiento,es_amueblado,es_penthouse,cuenta_con_cocina_integral,cuenta_con_sala,cuenta_con_closet,...,cuenta_con_terraza,cuenta_con_comedor,cuenta_con_area_de_lavado,cuenta_con_salon_usos_multiples,cuenta_con_mantenimiento_incluido,cuenta_con_vigilancia_24_horas,direccion,colonia,cp,municipio
0,24000.0,100,2.0,2.0,1.0,0,0,1,1,0,...,0,1,0,0,0,0,san jeronimo lidice san jeronimo lidice la mag...,san jeronimo lidice,10200,la magdalena contreras
1,12000.0,50,1.0,1.0,1.0,1,0,0,1,1,...,0,0,1,0,1,0,callejon del prado barrio san francisco la mag...,el prado,9480,la magdalena contreras
2,34100.0,232,3.0,4.0,2.0,0,0,1,1,1,...,1,1,0,0,0,1,blvd. adolfo ruiz cortines 2775 san jeronimo l...,adolfo ruiz cortines,4630,la magdalena contreras
3,16000.0,165,2.0,1.0,1.0,0,0,0,1,0,...,0,1,0,0,0,0,magnolia 26 san jeronimo lidice la magdalena c...,san jeronimo lidice,10200,la magdalena contreras
4,26000.0,180,3.0,2.0,2.0,0,0,1,0,1,...,0,0,0,0,0,1,san marcos 11 pedregal 2 la magdalena contreras,san marcos,2020,la magdalena contreras
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13537,14000.0,70,2.0,1.0,1.0,0,0,1,1,1,...,0,1,0,0,0,0,calzada lucio blanco 385 san juan tlihuaca azc...,san juan tlihuaca,2400,azcapotzalco
13538,14500.0,65,2.0,2.0,1.0,0,0,0,1,1,...,0,1,0,0,1,1,norte 77 3211 obrero popular obrero popular az...,obrero popular,2840,azcapotzalco
13539,9000.0,30,1.0,1.0,0.0,1,0,0,0,0,...,0,0,0,0,0,0,aquiles serdan 24 san francisco tetecala azcap...,san francisco tetecala,2730,azcapotzalco
13540,14000.0,56,2.0,1.0,1.0,1,0,1,1,0,...,1,1,0,0,0,1,sabino 578 col. ampliacion del gas alcaldia az...,del gas,2950,azcapotzalco


### Read AGEB data with colonia

In [6]:
data_folder = "../../data/processed/INEGI/colonia"
csv_files = [f for f in os.listdir(data_folder) if f.endswith('.csv')]
print("Archivos encontrados:", list(csv_files))

Archivos encontrados: ['20251002183836_iztapalapa.csv', '20250925224413_benito_juarez.csv', '20250926110422_cuauhtemoc.csv', '20250930100111_tlalpan.csv', '20250926043944_milpa_alta.csv', '20250926223742_venustiano_carranza.csv', '20250929081834_xochimilco.csv', '20250928025355_coyoacan.csv', '20250928113408_alvaro_obregon.csv', '20250926014033_cuajimalpa_de_morelos.csv', '20250926074342_la_magdalena_contreras.csv', '20250927030053_azcapotzalco.csv', '20250927085746_tlahuac.csv', '20251004073327_gustavo_a_madero.csv', '20250926182507_iztacalco.csv', '20250926143135_miguel_hidalgo.csv']


In [7]:
len(csv_files)

16

In [8]:
data = []
for file in csv_files:
    print(f"\nArchivo: {file}")
    df_tmp = pd.read_csv(os.path.join(data_folder, file), low_memory=False)
    data.append(df_tmp)


Archivo: 20251002183836_iztapalapa.csv

Archivo: 20250925224413_benito_juarez.csv

Archivo: 20250926110422_cuauhtemoc.csv

Archivo: 20250930100111_tlalpan.csv

Archivo: 20250926043944_milpa_alta.csv

Archivo: 20250926223742_venustiano_carranza.csv

Archivo: 20250929081834_xochimilco.csv

Archivo: 20250928025355_coyoacan.csv

Archivo: 20250928113408_alvaro_obregon.csv

Archivo: 20250926014033_cuajimalpa_de_morelos.csv

Archivo: 20250926074342_la_magdalena_contreras.csv

Archivo: 20250927030053_azcapotzalco.csv

Archivo: 20250927085746_tlahuac.csv

Archivo: 20251004073327_gustavo_a_madero.csv

Archivo: 20250926182507_iztacalco.csv

Archivo: 20250926143135_miguel_hidalgo.csv


In [9]:
df_ageb = pd.concat(data, ignore_index=True)

In [10]:
pat = re.compile(r"'neighbourhood':\s*'([^']+)'")
df_ageb['neighbourhood'] = df_ageb['address'].str.extract(pat)

In [11]:
df_ageb['neighbourhood'].isnull().sum()

32008

In [12]:
df_ageb.columns

Index(['ENTIDAD', 'NOM_ENT', 'MUN', 'NOM_MUN', 'LOC', 'NOM_LOC', 'AGEB', 'MZA',
       'POBTOT', 'POBFEM',
       ...
       'NOMREF3', 'lon', 'lat', 'municipio', 'address', 'road', 'quarter',
       'borough', 'postcode', 'neighbourhood'],
      dtype='object', length=135)

### Read localidades data

In [13]:
df_loc = pd.read_csv('../../data/raw/localidades/TR_LOCALIDAD_09 2.csv')

In [14]:
df_loc.columns

Index(['ENT', 'MUN', 'LOC', 'TIPOLOC', 'OTROTIPO_C', 'DIS_TRANS', 'AUTO',
       'MICRO', 'CAMION', 'TAXI',
       ...
       'USOIGL', 'USOFEST', 'USOCOMP', 'USOFAM', 'USOHOG', 'PROBLEMA',
       'PROBLEMA_O_C', 'POBTOT', 'TAMLOC', 'LOC_LENGUA40PC'],
      dtype='object', length=162)

In [15]:
df_loc.head()

,ENT,MUN,LOC,TIPOLOC,OTROTIPO_C,DIS_TRANS,AUTO,MICRO,CAMION,TAXI,...,USOIGL,USOFEST,USOCOMP,USOFAM,USOHOG,PROBLEMA,PROBLEMA_O_C,POBTOT,TAMLOC,LOC_LENGUA40PC
0,9,4,10,2,NaN,3,NaN,NaN,NaN,NaN,...,9.0,9.0,9.0,9.0,9.0,8,NaN,728,4,3
1,9,4,50,4,NaN,1,1.0,3.0,6.0,7.0,...,9.0,9.0,9.0,9.0,9.0,6,NaN,486,3,3
2,9,4,54,2,NaN,1,1.0,3.0,6.0,7.0,...,9.0,9.0,9.0,9.0,9.0,8,NaN,1233,5,3
3,9,4,55,2,NaN,1,2.0,3.0,6.0,7.0,...,NaN,NaN,NaN,NaN,NaN,6,NaN,80,1,3
4,9,4,56,7,NaN,3,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,6,NaN,108,2,3


In [16]:
df_loc.columns


Index(['ENT', 'MUN', 'LOC', 'TIPOLOC', 'OTROTIPO_C', 'DIS_TRANS', 'AUTO',
       'MICRO', 'CAMION', 'TAXI',
       ...
       'USOIGL', 'USOFEST', 'USOCOMP', 'USOFAM', 'USOHOG', 'PROBLEMA',
       'PROBLEMA_O_C', 'POBTOT', 'TAMLOC', 'LOC_LENGUA40PC'],
      dtype='object', length=162)

### Read colonias/cp file

In [17]:
df_cp = pd.read_csv('../../data/raw/CP/CPdescarga - Distrito_Federal.csv')

### Join INEGI files

In [18]:
df_loc.shape

(601, 162)

In [19]:
df_ageb.shape

(373150, 135)

In [20]:
set(df_ageb.columns).intersection(set(df_loc.columns))

{'LOC', 'MUN', 'POBTOT'}

In [21]:
df_loc[(df_loc.LOC == 10)&(df_loc.MUN == 4)]

,ENT,MUN,LOC,TIPOLOC,OTROTIPO_C,DIS_TRANS,AUTO,MICRO,CAMION,TAXI,...,USOIGL,USOFEST,USOCOMP,USOFAM,USOHOG,PROBLEMA,PROBLEMA_O_C,POBTOT,TAMLOC,LOC_LENGUA40PC
0,9,4,10,2,NaN,3,NaN,NaN,NaN,NaN,...,9.0,9.0,9.0,9.0,9.0,8,NaN,728,4,3


In [22]:
df_ageb[['LOC','MUN']]

,LOC,MUN
0,1,7
1,1,7
2,1,7
3,1,7
4,1,7
...,...,...
373145,1,16
373146,1,16
373147,1,16
373148,1,16


In [23]:
df_loc.LOC.value_counts()

LOC
140    4
112    4
113    4
135    3
198    3
      ..
344    1
345    1
348    1
352    1
293    1
Name: count, Length: 361, dtype: int64

In [24]:
df_ageb.LOC.value_counts()

LOC
1      349700
26       4234
11       3253
27       3066
21       2162
24       2108
20       2011
33       1400
17       1249
36       1009
29        774
152       584
15        425
135       421
19        288
300       271
110       195
Name: count, dtype: int64

In [25]:
df_ageb[(df_ageb.LOC == 10)]

,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,AGEB,MZA,POBTOT,POBFEM,...,NOMREF3,lon,lat,municipio,address,road,quarter,borough,postcode,neighbourhood


In [26]:
#df_inegi = df_ageb.merge(df_loc, on = ['LOC', 'MUN'])

### Join all files

In [27]:
df_web.colonia.value_counts()

colonia
hidalgo                         3924
cuauhtemoc                      2073
santa fe                         727
cuajimalpa                       502
hipodromo                        333
                                ... 
5 de mayo                          1
culhuacan ctm seccion v            1
Unidad Habitacional Rosedal        1
culhuacan ctm canal nacional       1
san francisco xocotitla            1
Name: count, Length: 413, dtype: int64

In [28]:
df_ageb.neighbourhood.value_counts()

neighbourhood
Colonia Agrícola Pantitlán                  2535
Colonia Cabeza de Juárez VI                 2047
Zona Escolar - Arbolillo - Jorge Negrete    1778
Unidad Habitacional Cabeza de Juárez        1561
2da Ampliación Santiago Acahualtepec        1469
                                            ... 
San Fernando                                   1
Lomas del Sol                                  1
El Ermitaño                                    1
Colonia Zona Arqueológica de Cuicuilco         1
Colonia San Gabriel                            1
Name: count, Length: 1905, dtype: int64

In [29]:
df_ageb['neighbourhood'] = df_ageb['neighbourhood'].apply(estandarizar_texto)
df_ageb['neighbourhood'] = df_ageb['neighbourhood'].str.replace('colonia ' , '').replace('Colonia ' , '')

In [30]:
df_ageb.neighbourhood.value_counts()

neighbourhood
                                            32008
agricola pantitlan                           2535
cabeza de juarez vi                          2047
zona escolar - arbolillo - jorge negrete     1778
jardin balbuena                              1573
                                            ...  
villa florence                                  1
lomas anahuac                                   1
san fernando                                    1
lomas del sol                                   1
san gabriel                                     1
Name: count, Length: 1873, dtype: int64

In [31]:
df_ageb[df_ageb.neighbourhood == 'hidalgo']

,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,AGEB,MZA,POBTOT,POBFEM,...,NOMREF3,lon,lat,municipio,address,road,quarter,borough,postcode,neighbourhood
216940,9,Ciudad de México,10,Álvaro Obregón,1,Álvaro Obregón,0031,1,85,45,...,Bondojito,2.792338e+06,825251.402850,alvaro_obregon,"{'road': 'Calle Ixmiquilpan', 'neighbourhood':...",Calle Ixmiquilpan,NaN,Álvaro Obregón,1120.0,hidalgo
216941,9,Ciudad de México,10,Álvaro Obregón,1,Álvaro Obregón,0031,1,85,45,...,Bejuco,2.792371e+06,825231.288991,alvaro_obregon,"{'house_number': '47', 'road': 'Calle Huichapa...",Calle Huichapan de León,NaN,Álvaro Obregón,1120.0,hidalgo
216942,9,Ciudad de México,10,Álvaro Obregón,1,Álvaro Obregón,0031,1,85,45,...,Ixmiquilpan,2.792343e+06,825205.283350,alvaro_obregon,"{'road': 'Calle Bondojito', 'neighbourhood': '...",Calle Bondojito,NaN,Álvaro Obregón,1120.0,hidalgo
216943,9,Ciudad de México,10,Álvaro Obregón,1,Álvaro Obregón,0031,1,85,45,...,Huichapan de León,2.792310e+06,825225.397150,alvaro_obregon,"{'road': 'Calle Bejuco', 'neighbourhood': 'Col...",Calle Bejuco,NaN,Álvaro Obregón,1120.0,hidalgo
216944,9,Ciudad de México,10,Álvaro Obregón,1,Álvaro Obregón,0031,2,85,48,...,Bondojito,2.792408e+06,825260.132800,alvaro_obregon,"{'road': 'Calle Ixmiquilpan', 'neighbourhood':...",Calle Ixmiquilpan,NaN,Álvaro Obregón,1120.0,hidalgo
216945,9,Ciudad de México,10,Álvaro Obregón,1,Álvaro Obregón,0031,2,85,48,...,Huichapan de León,2.792446e+06,825240.538733,alvaro_obregon,"{'road': 'Calle Florencio Miranda', 'neighbour...",Calle Florencio Miranda,NaN,Álvaro Obregón,1120.0,hidalgo
216946,9,Ciudad de México,10,Álvaro Obregón,1,Álvaro Obregón,0031,2,85,48,...,Ixmiquilpan,2.792416e+06,825212.399346,alvaro_obregon,"{'house_number': '295', 'road': 'Calle Bondoji...",Calle Bondojito,NaN,Álvaro Obregón,1120.0,hidalgo
216947,9,Ciudad de México,10,Álvaro Obregón,1,Álvaro Obregón,0031,2,85,48,...,Florencio Miranda,2.792378e+06,825231.992854,alvaro_obregon,"{'house_number': '47', 'road': 'Calle Huichapa...",Calle Huichapan de León,NaN,Álvaro Obregón,1120.0,hidalgo
216949,9,Ciudad de México,10,Álvaro Obregón,1,Álvaro Obregón,0031,3,160,92,...,Tulancingo,2.792420e+06,825204.064550,alvaro_obregon,"{'house_number': '295', 'road': 'Calle Bondoji...",Calle Bondojito,NaN,Álvaro Obregón,1120.0,hidalgo
216950,9,Ciudad de México,10,Álvaro Obregón,1,Álvaro Obregón,0031,3,160,92,...,Bondojito,2.792425e+06,825143.198428,alvaro_obregon,"{'road': 'Calle Tulancingo', 'neighbourhood': ...",Calle Tulancingo,NaN,Álvaro Obregón,1120.0,hidalgo


In [32]:
context = ['ENTIDAD', 'NOM_ENT', 'MUN', 'NOM_MUN', 'LOC', 'NOM_LOC', 'AGEB', 'MZA',
            'key', 'ENTIDAD_fm', 'MUN_fm', 'LOC_fm', 'AGEB_fm', 'MZA_fm', 'CVEVIAL',
            'CVESEG', 'CVEFT', 'NOMVIAL', 'TIPOVIAL', 'CVEVIAL1', 'CVESEG1',
            'CVEREF1', 'TIPOVR1', 'NOMREF1', 'CVEVIAL2', 'CVESEG2', 'CVEREF2',
            'TIPOVR2', 'NOMREF2', 'CVEVIAL3', 'CVESEG3', 'CVEREF3', 'TIPOVR3',
            'NOMREF3', 'lon', 'lat', 'municipio', 'address', 'road', 'quarter',
            'borough', 'postcode', 'neighbourhood']

In [33]:
cols = list(set(df_ageb.columns)-set(context))

In [34]:
df_ageb.columns[100:]

Index(['key', 'ENTIDAD_fm', 'MUN_fm', 'LOC_fm', 'AGEB_fm', 'MZA_fm', 'CVEVIAL',
       'CVESEG', 'CVEFT', 'NOMVIAL', 'TIPOVIAL', 'CVEVIAL1', 'CVESEG1',
       'CVEREF1', 'TIPOVR1', 'NOMREF1', 'CVEVIAL2', 'CVESEG2', 'CVEREF2',
       'TIPOVR2', 'NOMREF2', 'CVEVIAL3', 'CVESEG3', 'CVEREF3', 'TIPOVR3',
       'NOMREF3', 'lon', 'lat', 'municipio', 'address', 'road', 'quarter',
       'borough', 'postcode', 'neighbourhood'],
      dtype='object')

In [35]:
df_ageb[cols] = df_ageb[cols].replace('*',0)
df_ageb[cols] = df_ageb[cols].replace('N/D',0)

In [36]:
df_ageb[cols] = df_ageb[cols].astype('float64')

In [37]:
df_ageb = df_ageb[df_ageb.neighbourhood != '']

In [38]:
df_ageb_grp = df_ageb.groupby('neighbourhood')[cols].mean().reset_index()

In [39]:
df_ageb_grp.shape

(1872, 93)

In [40]:
df_ageb_grp

,neighbourhood,TOTHOG,VIVTOT,VPH_STVP,VPH_HMICRO,P_60YMAS_F,VPH_LETR,PHOGJEF_F,VPH_SNBIEN,P_18YMAS_F,...,VPH_PC,VPH_AEASP,VIVPAR_UT,TVIVPAR,VPH_3YMASC,P_18YMAS,POB65_MAS,VPH_TV,PROM_OCUP,POBTOT
0,10 de mayo,39.000000,44.833333,15.583333,25.833333,12.333333,0.0,68.500000,0.000000,50.833333,...,23.916667,38.916667,2.750000,44.583333,35.416667,95.666667,14.416667,37.166667,2.934167,124.500000
1,12 de diciembre,107.478873,110.633803,55.084507,59.338028,30.492958,0.0,124.014085,0.000000,146.704225,...,49.718310,106.704225,0.000000,109.915493,75.014085,273.746479,36.802817,101.943662,3.459155,362.830986
2,15 de agosto,110.775862,124.163793,55.336207,67.068966,34.827586,0.0,141.431034,0.000000,155.456897,...,56.094828,110.387931,2.689655,115.887931,85.896552,288.448276,41.827586,107.163793,3.031897,384.879310
3,16 de septiembre,123.930233,137.441860,70.953488,89.325581,43.953488,0.0,171.348837,0.000000,174.697674,...,83.883721,123.767442,4.372093,124.651163,97.232558,334.255814,49.744186,119.069767,3.270465,414.046512
4,19 de mayo,48.879032,52.661290,25.064516,34.774194,13.411290,0.0,74.040323,0.000000,73.564516,...,26.508065,48.709677,0.459677,47.854839,40.153226,142.750000,16.580645,47.233871,3.401855,189.056452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1867,zona urbana ejidal san andres tomatlan,71.927711,74.825301,27.108434,42.590361,20.192771,0.0,113.132530,0.000000,112.903614,...,38.078313,71.578313,0.228916,73.506024,52.415663,212.271084,24.728916,68.746988,3.978554,279.403614
1868,zona urbana ejido santa maria aztahuacan,46.722892,49.080895,15.179002,24.079174,10.822719,0.0,65.514630,0.215146,65.006885,...,19.736661,46.502582,0.165232,47.786575,34.358003,123.518072,12.808950,44.810671,3.518055,171.593804
1869,zona urbana los reyes culhuacan,54.843854,61.252492,18.392027,32.232558,12.754153,0.0,71.857143,0.089701,77.411960,...,26.152824,54.544850,0.186047,60.863787,39.073090,149.501661,15.026578,52.348837,3.598904,198.634551
1870,zona urbana santa martha acatitla norte,58.078853,59.935484,17.802867,30.501792,15.627240,0.0,63.888889,0.071685,81.336918,...,26.204301,57.888889,0.043011,57.824373,40.286738,154.691756,19.910394,55.616487,3.162222,204.573477


In [41]:
df_ageb_grp.neighbourhood.values

array(['10 de mayo', '12 de diciembre', '15 de agosto', ...,
       'zona urbana los reyes culhuacan',
       'zona urbana santa martha acatitla norte',
       'zona urbana santha martha acatitla sur'], dtype=object)

In [42]:
df_merge = df_web.merge(df_ageb_grp, left_on='colonia', right_on='neighbourhood', how='left', indicator=True)

In [43]:
df_merge.shape

(13542, 119)

In [44]:
df_merge.columns

Index(['precio_mxn', 'lote_m2', 'recamaras', 'baños', 'estacionamiento',
       'es_amueblado', 'es_penthouse', 'cuenta_con_cocina_integral',
       'cuenta_con_sala', 'cuenta_con_closet',
       ...
       'VPH_AEASP', 'VIVPAR_UT', 'TVIVPAR', 'VPH_3YMASC', 'P_18YMAS',
       'POB65_MAS', 'VPH_TV', 'PROM_OCUP', 'POBTOT', '_merge'],
      dtype='object', length=119)

In [45]:
df_merge._merge.value_counts()

_merge
both          11868
left_only      1674
right_only        0
Name: count, dtype: int64

In [46]:
df_merge[df_merge._merge == 'left_only'][['colonia', 'neighbourhood']]

,colonia,neighbourhood
0,san jeronimo lidice,NaN
3,san jeronimo lidice,NaN
5,san nicolas totolapan,NaN
6,san jeronimo lidice,NaN
8,san jeronimo lidice,NaN
...,...,...
13502,Lindavista,NaN
13503,lindavista sur,NaN
13524,centro de azcapotzalco,NaN
13528,el jaguey,NaN


In [47]:
df_merge[df_merge._merge == 'left_only']['colonia'].value_counts()

colonia
bosque                          209
polanco v seccion               182
tlalpan                         154
polanco iv seccion              131
del valle sur                    89
                               ... 
culhuacan ctm canal nacional      1
Unidad Habitacional Rosedal       1
culhuacan ctm seccion v           1
FOVISSSTE Oxtopulco               1
loma la palma                     1
Name: count, Length: 127, dtype: int64

In [48]:
df_merge = df_merge[df_merge._merge == 'both']

In [49]:
len(cols)

92

In [50]:
df_merge.columns[:50]

Index(['precio_mxn', 'lote_m2', 'recamaras', 'baños', 'estacionamiento',
       'es_amueblado', 'es_penthouse', 'cuenta_con_cocina_integral',
       'cuenta_con_sala', 'cuenta_con_closet', 'cuenta_con_balcon',
       'cuenta_con_gimnasio', 'cuenta_con_alberca', 'cuenta_con_elevador',
       'cuenta_con_roof_garden', 'cuenta_con_terraza', 'cuenta_con_comedor',
       'cuenta_con_area_de_lavado', 'cuenta_con_salon_usos_multiples',
       'cuenta_con_mantenimiento_incluido', 'cuenta_con_vigilancia_24_horas',
       'direccion', 'colonia', 'cp', 'municipio', 'neighbourhood', 'TOTHOG',
       'VIVTOT', 'VPH_STVP', 'VPH_HMICRO', 'P_60YMAS_F', 'VPH_LETR',
       'PHOGJEF_F', 'VPH_SNBIEN', 'P_18YMAS_F', 'VPH_EXCSA', 'VPH_TINACO',
       'VPH_CISTER', 'VPH_AUTOM', 'P_15A49_F', 'VPH_TELEF', 'P_6A11_F',
       'VPH_SINCINT', 'P_12A14_F', 'P_8A14_M', 'VPH_C_SERV', 'PRO_OCUP_C',
       'P_0A2_F', 'VPH_SINTIC', 'VPH_S_ELEC'],
      dtype='object')

In [51]:
df_merge.to_parquet('../../data/processed/merged_inmuebles24_departamentos_20251005.parquet', index=False)